In [1]:
import os
import openai
import sys
import argparse
import logging
import torch
import random
import time
import json
import re
import numpy as np
import pandas as pd

In [2]:
human = pd.read_csv('./third_party/data/mturk_result/Batch_results_develop.csv')
human = human.loc[human['ApprovalTime'].dropna().index].reset_index(drop=True)
human = human[['Input.dataset', 'Input.task', 'Input.Speaker', 'Answer.Appropriateness_llama7b', 'Answer.Appropriateness_llama13b', 'Answer.Appropriateness_vicuna7b', 'Answer.Appropriateness_vicuna13b', 'Answer.Style_llama7b', 'Answer.Style_llama13b', 'Answer.Style_vicuna7b', 'Answer.Style_vicuna13b']]

In [3]:
human[human['Input.task']=='Negative'][['Answer.Appropriateness_llama7b', 'Answer.Appropriateness_llama13b', 'Answer.Appropriateness_vicuna7b', 'Answer.Appropriateness_vicuna13b', 'Answer.Style_llama7b', 'Answer.Style_llama13b', 'Answer.Style_vicuna7b', 'Answer.Style_vicuna13b']].mean()

Answer.Appropriateness_llama7b      51.13
Answer.Appropriateness_llama13b     54.25
Answer.Appropriateness_vicuna7b     59.49
Answer.Appropriateness_vicuna13b    71.08
Answer.Style_llama7b                66.03
Answer.Style_llama13b               68.31
Answer.Style_vicuna7b               66.53
Answer.Style_vicuna13b              62.46
dtype: float64

In [4]:
with open('./third_party/output_dir_corr/DD_Formal/response_form_result_DD_5shot_llama7b/test.1.json') as user_file:
    parsed_json = json.load(user_file)
data = parsed_json['instances'][0:50]
DD_input_li = [item['input'] for item in data]

with open('./third_party/output_dir_corr/BST_Formal/response_form_result_BST_5shot_llama7b/test.1.json') as user_file:
    parsed_json = json.load(user_file)
data = parsed_json['instances'][0:50]
BST_input_li = [item['input'] for item in data]

In [5]:
dataset_li = ['DD', 'BST']
task_li = ['Formal', 'Informal', 'Positive', 'Negative']
model_li = ['llama7b','llama13b','vicuna7b','vicuna13b']
# auto_li = ['NLL','BLEU','SBERT','Style Strength']
auto_li = ['NLL','ChatGPT','BLEU','SBERT','Style Strength']
# appro_auto_li = ['NLL','BLEU','SBERT']
appro_auto_li = ['NLL','ChatGPT','BLEU','SBERT']
# style_auto_li = ['Style Strength']
style_auto_li = ['Style Strength']

In [6]:
root_path = './third_party/output_dir_corr'


df_li = []
for dataset in dataset_li:
    for task in task_li:
        dir_path = root_path+'/'+dataset+'_'+task

        input_li = DD_input_li if dataset=='DD' else BST_input_li
        auto_df = pd.DataFrame(index=input_li)
        auto_df.index.name = 'Input.Speaker'
        for auto in auto_li:
            for model in model_li:
                auto_df[f'Auto.{auto}_{model}'] = np.nan
                
        for file_name in os.listdir(dir_path):
            model = file_name.split('_')[-1]
            file_path = dir_path + '/' + file_name + '/test.1.json'
            with open(file_path) as user_file:
                parsed_json = json.load(user_file)
            data_li = parsed_json['instances'][0:50]
            for data in data_li:
                add_data = []
                for auto in auto_li:
                    add_data.append(data[auto])
                auto_df.loc[data['input'],[f'Auto.{auto}_{model}' for auto in auto_li]] = add_data
        auto_df = auto_df.reset_index()
        auto_df['Input.dataset'] = dataset
        auto_df['Input.task'] = task  
        df_li.append(auto_df)
        

In [7]:
auto_df_total = pd.concat(df_li)
auto_df_total['Input.Speaker'] = auto_df_total['Input.Speaker'].apply(lambda x: x.strip())
human['Input.Speaker'] = human['Input.Speaker'].apply(lambda x: x.strip())
data_total = pd.merge(human, auto_df_total, how='left', on=['Input.task','Input.Speaker','Input.dataset'])
data_total_bloom = data_total.copy()

## Correlation Result

In [8]:
import warnings
warnings.filterwarnings("ignore")

print('Sample level')
print('Pearson Correlation\n')


for task in task_li:
    print('task: ', task)
    
    data_tmp = data_total[(data_total['Input.task']==task)]
    data_tmp_human = data_tmp[[f'Answer.Appropriateness_{model}' for model in model_li]]
    data_tmp_human_style = data_tmp[[f'Answer.Style_{model}' for model in model_li]]
    for auto in appro_auto_li:
        data_tmp_auto = data_tmp[[f'Auto.{auto}_{model}' for model in model_li]]
        corr_li = []
        for i in data_tmp_human.index:
            corr_li.append(np.corrcoef(data_tmp_human.loc[i].values, data_tmp_auto.loc[i].values)[0,-1])

        print(f'{auto}: ', np.array(corr_li)[~np.isnan(corr_li)].mean()*100)

    for auto in style_auto_li:
        data_tmp_auto_style = data_tmp[[f'Auto.{auto}_{model}' for model in model_li]]
        corr_li = []
        for i in data_tmp_human_style.index:
            corr_li.append(np.corrcoef(data_tmp_human_style.loc[i].values, data_tmp_auto_style.loc[i].values)[0,-1])
        print(f'{auto}: ', np.array(corr_li)[~np.isnan(corr_li)].mean()*100)
    
    print('\n')

Sample level
Pearson Correlation

task:  Formal
NLL:  34.233441246299265
ChatGPT:  27.042071644474895
BLEU:  -11.363660021965126
SBERT:  11.274251997683718
Style Strength:  36.57993489868076


task:  Informal
NLL:  11.572363040780909
ChatGPT:  21.32424398022153
BLEU:  1.5084893375040096
SBERT:  8.216736983094501
Style Strength:  22.471223101388873


task:  Positive
NLL:  27.478064038983145
ChatGPT:  28.928151724795114
BLEU:  11.38463514144536
SBERT:  20.01424738232686
Style Strength:  16.63982702751144


task:  Negative
NLL:  42.05501922357687
ChatGPT:  15.234138082058072
BLEU:  5.104175245314683
SBERT:  28.665750094310464
Style Strength:  32.106553670387115




In [10]:
import warnings
warnings.filterwarnings("ignore")

from scipy import stats
print('Sample level')
print('Kendall Correlation\n')

for task in task_li:
    print('task: ', task)
    
    data_tmp = data_total[(data_total['Input.task']==task)]
    
    data_tmp_human = data_tmp[[f'Answer.Appropriateness_{model}' for model in model_li]]
    data_tmp_human_style = data_tmp[[f'Answer.Style_{model}' for model in model_li]]
    for auto in appro_auto_li:
        data_tmp_auto = data_tmp[[f'Auto.{auto}_{model}' for model in model_li]]
        corr_li = []
        for i in data_tmp_human.index:
            corr_li.append(stats.kendalltau(data_tmp_human.loc[i].values, data_tmp_auto.loc[i].values).correlation)

        print(f'{auto}: ', np.array(corr_li)[~np.isnan(corr_li)].mean()*100)

    for auto in style_auto_li:
        data_tmp_auto_style = data_tmp[[f'Auto.{auto}_{model}' for model in model_li]]
        corr_li = []
        for i in data_tmp_human.index:
            corr_li.append(stats.kendalltau(data_tmp_human_style.loc[i].values, data_tmp_auto_style.loc[i].values).correlation)

        print(f'{auto}: ', np.array(corr_li)[~np.isnan(corr_li)].mean()*100)
    
    print('\n')

Sample level
Kendall Correlation

task:  Formal
NLL:  25.263139397475072
ChatGPT:  20.717839009927143
BLEU:  -6.08204072994886
SBERT:  7.327648540982666
Style Strength:  32.18723414973005


task:  Informal
NLL:  18.973762571576177
ChatGPT:  17.367521942103124
BLEU:  4.9328488025747
SBERT:  6.224966432952758
Style Strength:  16.751278536927096


task:  Positive
NLL:  28.261634470233364
ChatGPT:  26.06305820660098
BLEU:  7.78660347623573
SBERT:  20.17197400550434
Style Strength:  17.800893814396186


task:  Negative
NLL:  34.0884155340145
ChatGPT:  16.44967746189785
BLEU:  3.1249875822516313
SBERT:  26.41805636871669
Style Strength:  19.137031629735823


